In [40]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

box_DIR = "../data/box/"
img_DIR_T = "../data/images/teams/"
fig_DIR = "../figs/teams/"

In [44]:
dfa = []
for year in range(2019,2024):
    df1 = pd.read_parquet(box_DIR + f"NBA_Box_T_Base_{year}.parquet")
    df1.insert(0,"GAME_ID",df1.pop("GAME_ID"))
    df4 = df1.groupby("GAME_ID")
    df4_1 = df4.nth(0)
    df4_2 = df4.nth(1)
    df4_1.columns = ["GAME_ID"] + [s + "1" for s in df4_1.columns if s != "GAME_ID"]
    df4_2.columns = ["GAME_ID"] + [s + "2" for s in df4_2.columns if s != "GAME_ID"]
    df4_3 = pd.merge(df4_1, df4_2, on="GAME_ID")
    df4_4 = df4.nth(1)
    df4_5 = df4.nth(0)
    df4_4.columns = ["GAME_ID"] + [s + "1" for s in df4_4.columns if s != "GAME_ID"]
    df4_5.columns = ["GAME_ID"] + [s + "2" for s in df4_5.columns if s != "GAME_ID"]
    df4_6 = pd.merge(df4_4, df4_5, on="GAME_ID")
    df5 = pd.concat([df4_3,df4_6])
    df5["FT_Diff"] = df5["FTA1"] - df5["FTA2"]
    df6 = pd.DataFrame(df5.groupby("TEAM_NAME1")["FT_Diff"].sum()).reset_index()
    df6.columns = ["Team","FT_Diff"]
    df6["Season"] = year + 1
    dfa.append(df6)


In [45]:
df7 = pd.concat(dfa)
df7 = df7.sort_values("FT_Diff",ascending=False).reset_index(drop=True).reset_index()
df7["index"] += 1  
df7["T"]= df7["Team"]
df7.insert(1,"T",df7.pop("T"))
df7.insert(3,"Season",df7.pop("Season"))
df8 = df7.head(10)

In [47]:
t = (
    GT(df8)
    .tab_header(
        title=md("Highest FT Differential, Past 5 Seasons"),
        # subtitle="Daily measurements in New York City (May 1-10, 1973)"
    )
    .tab_source_note(source_note="@SravanNBA | source: nba.com/stats via nba_api")
    .cols_label(
        index = "#",
        T = "",
        FT_Diff = "FT Diff",
    )
    .fmt_image(
        columns="T",
        path = img_DIR_T,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="Team")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        table_margin_left = 1,
        table_margin_right = 1,
    )
)
t.save(fig_DIR + "ft_diff.png",scale=3,expand=2,web_driver="edge")
t

GT(_tbl_data=   index                      T                   Team  Season  FT_Diff
0      1     Los Angeles Lakers     Los Angeles Lakers    2024      507
1      2     Los Angeles Lakers     Los Angeles Lakers    2023      476
2      3  Oklahoma City Thunder  Oklahoma City Thunder    2020      379
3      4   New Orleans Pelicans   New Orleans Pelicans    2021      362
4      5              Utah Jazz              Utah Jazz    2022      335
5      6        Milwaukee Bucks        Milwaukee Bucks    2020      249
6      7        Milwaukee Bucks        Milwaukee Bucks    2022      247
7      8         Boston Celtics         Boston Celtics    2024      232
8      9             Miami Heat             Miami Heat    2024      229
9     10       Sacramento Kings       Sacramento Kings    2022      226, _body=<great_tables._gt_data.Body object at 0x000001C472A5D410>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.default: 1>, column_label='#', column_align='center', column_width=None), ColInfo(var='T', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='center', column_width=None), ColInfo(var='Team', type=<ColInfoTypeEnum.default: 1>, column_label='Team', column_align='left', column_width=None), ColInfo(var='Season', type=<ColInfoTypeEnum.default: 1>, column_label='Season', column_align='center', column_width=None), ColInfo(var='FT_Diff', type=<ColInfoTypeEnum.default: 1>, column_label='FT Diff', column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=6, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=7, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=8, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=9, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title=Text(text='Highest FT Differential, Past 5 Seasons', type='from_markdown'), subtitle=None, preheader=None), _stubhead=None, _source_notes=['@SravanNBA | source: nba.com/stats via nba_api'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001C472A60E10>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000001C4716E6FD0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value=1), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value=1), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='floralwhite'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['Consolas']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value=10), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolea